In [3]:
from google.colab import files

uploaded = files.upload()

Saving dicionario_de_dados.docx to dicionario_de_dados.docx
Saving doencas_respiratorias_cronicas.pdf to doencas_respiratorias_cronicas.pdf


In [4]:
# 📍 Instalar as dependências necessárias
%pip install transformers sentencepiece python-docx pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 62.7 MB/s eta 0:00:00


In [5]:
# 📚 Imports
import fitz  # pymupdf
import docx
from transformers import pipeline
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
# 📂 Função para carregar e extrair texto de arquivos
def extract_text_from_docx(path):
    import docx
    doc = docx.Document(path)
    full_text = []

    # Extrair parágrafos comuns
    for p in doc.paragraphs:
        if p.text.strip():
            full_text.append(p.text.strip())

    # Extrair conteúdo das tabelas
    for table in doc.tables:
        for row in table.rows:
            row_text = [cell.text.strip() for cell in row.cells if cell.text.strip()]
            if row_text:
                full_text.append(" | ".join(row_text))

    return "\n".join(full_text)

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    return "\n".join([page.get_text() for page in doc])

In [7]:
# 📄 Carregando arquivos (substitua os paths pelos seus uploads)
docx_path = "/content/dicionario_de_dados.docx"
pdf_path = "/content/doencas_respiratorias_cronicas.pdf"

# 📄 Extraindo conteúdo dos arquivos
docx_text = extract_text_from_docx(docx_path)
pdf_text = extract_text_from_pdf(pdf_path)

In [8]:
# Modelos Hugging Face
model_names = [
    "deepset/roberta-base-squad2",
    "distilbert-base-cased-distilled-squad",
    "ktrapeznikov/albert-xlarge-v2-squad-v2"
]
qa_pipelines = {
    name: pipeline("question-answering", model=name, tokenizer=name)
    for name in model_names
}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/235M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

Device set to use cuda:0


In [2]:
# Separar perguntas e respostas por fonte
docx_questions = [
    "Na tabela FCESGEST, quais os domínios do campo INDTIPOCAD?",
    "Na tabela LFCES004, qual a descrição do campo PFPJ_IND",
    "Na tabela LFCES035, qual nme do campo responsável por armazenar a Data do Processamento e seu tipo?"
]

pdf_questions = [
    "Com quais doenças respiratórias o baqueteamento digital pode estar associado?",
    "Quais são os principais sintomas da conjuntivite e rinite alérgica?",
    "Qual é o papel da lavagem nasal com solução salina no tratamento da rinite alérgica?"
]

respostas_esperadas = {
    # PDF
    pdf_questions[0]: "Doenças cardiovasculares, digestivas e pulmonares.",
    pdf_questions[1]: "Prurido, lacrimejamento, sensação de corpo estranho no olho, fotofobia, hiperemia e edema periocular.",
    pdf_questions[2]: "Aliviar a irritação tecidual, umedecer a mucosa e auxiliar na remoção de secreções, aliviando temporariamente a obstrução nasal e melhorando o olfato.",
    # DOCX
    docx_questions[0]: "1 - Completo\n2 - Só Profissionais",
    docx_questions[1]: "Indica se é Pessoa Física ou Jurídica",
    docx_questions[2]: "DT_PROCESS - DATE",
}

In [9]:
def calcular_similaridade(a, b):
    vect = TfidfVectorizer().fit([a, b])
    tfidf = vect.transform([a, b])
    return round(cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0], 3)

In [ ]:
# 🔍 Função para buscar o trecho mais relevante no texto
def get_best_context(question, text_blocks, top_k=1):
    vect = TfidfVectorizer().fit(text_blocks + [question])
    tfidf_matrix = vect.transform(text_blocks + [question])
    sims = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]
    top_indices = sims.argsort()[-top_k:][::-1]
    return " ".join([text_blocks[i] for i in top_indices])

# Quebrar os textos em blocos menores para busca
def chunk_text(text, chunk_size=300):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

def criar_prompt(pergunta, contexto):
    return f"""
    Você é um especialista e deve responder APENAS com base no texto abaixo.
    Responda de forma objetiva, em uma frase curta.

    Pergunta: {pergunta}
    """

In [22]:
docx_chunks = chunk_text(docx_text, chunk_size=300)
pdf_chunks = chunk_text(pdf_text, chunk_size=300)

In [47]:
def ask_questions_with_comparison_v2():
    results = []
    for source, questions, chunks in [
        ("DOCX", docx_questions, docx_chunks),
        ("PDF", pdf_questions, pdf_chunks)
    ]:
        for q in questions:
            context = get_best_context(q, chunks)
            prompt = criar_prompt(q, context)  # aplica a estratégia de prompt

            # Contexto enviado ao modelo com instrução + trecho
            contexto_prompt = f"""
            Baseie-se APENAS no texto abaixo para responder.
            Se não encontrar a resposta, escreva "Não encontrado".

            Texto:
            \"\"\"{context}\"\"\"
            """

            resposta_esperada = respostas_esperadas.get(q, "")
            for model_name, nlp in qa_pipelines.items():
                try:
                    answer = nlp(question=q, context=contexto_prompt)
                    similaridade = calcular_similaridade(answer["answer"], resposta_esperada)

                    results.append({
                        "Fonte": source,
                        "Pergunta": q,
                        "Modelo": model_name,
                        "Resposta do Modelo": answer["answer"],
                        "Resposta Esperada": resposta_esperada,
                        "Score do Modelo": round(answer["score"], 3),
                        "Similaridade com Esperada": similaridade,
                    })
                except Exception as e:
                    results.append({
                        "Fonte": source,
                        "Pergunta": q,
                        "Modelo": model_name,
                        "Resposta do Modelo": str(e),
                        "Resposta Esperada": resposta_esperada,
                        "Score do Modelo": 0,
                        "Similaridade com Esperada": 0.0
                    })
    return results

In [50]:
results = ask_questions_with_comparison_v2()
df_results = pd.DataFrame(results)

# 📊 Juntar os resultados e exibir em uma tabela
pd.set_option('display.max_colwidth', None)

# Mostrar os resultados ordenados por pergunta
df_results.sort_values(by=["Fonte", "Pergunta", "Modelo"], inplace=True)
df_results

,Fonte,Pergunta,Modelo,Resposta,Resposta Esperada,Score,Similaridade com Esperada
0,DOCX,"Na tabela FCESGEST, quais os domínios do campo INDTIPOCAD?",deepset/roberta-base-squad2,\n,1 - Completo\n2 - Só Profissionais,0.001,0.000
1,DOCX,"Na tabela FCESGEST, quais os domínios do campo INDTIPOCAD?",distilbert-base-cased-distilled-squad,QUIMIOTERAPIA,1 - Completo\n2 - Só Profissionais,0.000,0.000
2,DOCX,"Na tabela FCESGEST, quais os domínios do campo INDTIPOCAD?",ktrapeznikov/albert-xlarge-v2-squad-v2,TABELA DE REPRESENTANTE LEGAL,1 - Completo\n2 - Só Profissionais,0.000,0.000
3,DOCX,"Na tabela LFCES004, qual a descrição do campo PFPJ_IND",deepset/roberta-base-squad2,"""Não encontrado"".",Indica se é Pessoa Física ou Jurídica,0.007,0.000
4,DOCX,"Na tabela LFCES004, qual a descrição do campo PFPJ_IND",distilbert-base-cased-distilled-squad,Se não encontrar a resposta,Indica se é Pessoa Física ou Jurídica,0.000,0.115
5,DOCX,"Na tabela LFCES004, qual a descrição do campo PFPJ_IND",ktrapeznikov/albert-xlarge-v2-squad-v2,"15""""""",Indica se é Pessoa Física ou Jurídica,0.000,0.000
6,DOCX,"Na tabela LFCES035, qual nme do campo responsável por armazenar a Data do Processamento e seu tipo?",deepset/roberta-base-squad2,"-""""""\n",DT_PROCESS - DATE,0.001,0.000
7,DOCX,"Na tabela LFCES035, qual nme do campo responsável por armazenar a Data do Processamento e seu tipo?",distilbert-base-cased-distilled-squad,COOPERATIVAS,DT_PROCESS - DATE,0.082,0.000
8,DOCX,"Na tabela LFCES035, qual nme do campo responsável por armazenar a Data do Processamento e seu tipo?",ktrapeznikov/albert-xlarge-v2-squad-v2,RL_COOPERATIVA,DT_PROCESS - DATE,0.000,0.000
9,PDF,Com quais doenças respiratórias o baqueteamento digital pode estar associado?,deepset/roberta-base-squad2,"cardiovasculares, digestivas e pulmonares","Doenças cardiovasculares, digestivas e pulmonares.",0.487,0.777
